<h2> 0. 데이터 로드 

In [28]:
!unzip /Users/sungheui/코딩 테스트/ArtClassify/train.zip -d ./

unzip:  cannot find or open /Users/sungheui/코딩, /Users/sungheui/코딩.zip or /Users/sungheui/코딩.ZIP.


In [30]:
cd /Users/sungheui/sh/lotteAI

/Users/sungheui/sh/lotteAI


In [31]:
#환경 설정 확인하기
import torch
print(f"mps 사용 가능 여부: {torch.backends.mps.is_available()}")
print(f"mps 지원 환경 여부: {torch.backends.mps.is_built()}")

#GPU check DEVICE
# print(f'PyTorch Version : [{torch.__version__}]')
# device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print(f'Device : [{device}]')

mps 사용 가능 여부: True
mps 지원 환경 여부: True


<h3> 2. 라이브러리 import 

In [32]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from tqdm.auto import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models
from glob import glob
import cv2
from PIL import Image
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

<h3> 3. Config setting

In [33]:
#CONFIG
torch.manual_seed(128)
BATCH_SIZE= 2
EPOCHS=40
LEARNING_RATE=1e-3
print(f'PyTorch Version : [{torch.__version__}]')
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
# device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

PyTorch Version : [2.1.2]
Device: mps


<h3> 4. Custom Datasets

In [34]:
# import pdb;pdb.set_trace() 
class LotteDataset(Dataset):
  def __init__(self, data_root, train_mode, transform=None):
    super(LotteDataset, self).__init__()
    self.train_mode=train_mode
    self.transform = transform
    self.label2idx = {'dog': 0,
             'elephant': 1,
             'giraffe': 2,
             'guitar': 3,
             'horse': 4,
             'house': 5,
             'person': 6}
   
    if self.train_mode==False:
        
        self.img_list= []
        img_list = []
        for file in os.listdir(data_root):
            file_root = os.path.join(data_root, file)
            for data in os.listdir(file_root):
                if '.jpg' in data:
                    data_path = os.path.join(file_root, data)
                    img_list.append(data_path)
                
        self.img_list = sorted(img_list)

    else: #학습할 때 
        self.img_list = []
        self.label_list=[]
        img_list = []
        for file in os.listdir(data_root):
            file_root = os.path.join(data_root, file)
            for data in os.listdir(file_root):
                if '.jpg' in data:
                    data_path = os.path.join(file_root, data)
                    img_list.append(data_path)

        self.img_list = sorted(img_list)
       
        for label in self.img_list:
            label = label.split('/')[-2]
           
            self.label_list.append(self.label2idx[label]) 
            

  def __len__(self):
      return len(self.img_list)

  def __getitem__(self, index):
    img_path = self.img_list[index]

    #Image, Label Loading
    if self.train_mode:
        label = self.label_list[index]
     
    img = Image.open(img_path)

    #Augmentation  
    if self.transform: 
      img = self.transform(img)

    if self.train_mode:
      return img,label
    else:
      return img

# if __name__=='__main__':
    # data = LotteDataset('./dataset/test',train_mode=False,transform=train_transforms)
    # data = LotteDataset('./dataset/train',train_mode=True,transform=train_transforms)
    
    
    
    
    # q= data.__getitem__(1)
    # print(label)


In [7]:
class MapTransform(Dataset):
    def __init__(self, dataset, transform, train_mode):
        self.dataset = dataset
        self.transform=transform
        self.train_mode=train_mode

    def __getitem__(self, index):
        if self.train_mode:
          return self.transform(self.dataset[index][0]), self.dataset[index][1]
        else:
          return self.transform(self.dataset[index])

    def __len__(self):
        return len(self.dataset)

In [35]:
#Train Augmentation 
train_transforms=transforms.Compose([
    transforms.RandomChoice([
        transforms.ColorJitter(brightness=(1,1.1)),
        transforms.ColorJitter(contrast=0.1), 
        transforms.ColorJitter(saturation=0.1),
    ]),
    transforms.RandomChoice([
        transforms.RandomAffine(degrees=15, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=10,fill=255),
        transforms.RandomCrop((224,224)), #이미지 사이즈 설정 
    ]),
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
#Test Augmentation 
test_transforms=transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((224,224)),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

In [11]:
#그냥 train-test -> validation x
train_data=LotteDataset('./dataset/train',train_mode=True)
test_data=LotteDataset('./dataset/test',train_mode=False)

trans_train_data=MapTransform(train_data,train_transforms,train_mode=True)
trans_test_data=MapTransform(test_data,test_transforms,train_mode=False)

train_iter=DataLoader(trans_train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
test_iter=DataLoader(trans_test_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=0)

In [36]:
##cross-validation 할때 이걸로 
all_data=LotteDataset('./dataset/train',train_mode=True,transform=train_transforms)
test_data=LotteDataset('./dataset/test',train_mode=False,transform=test_transforms)
test_iter=DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=0)

In [37]:
print(len(trans_train_data)) #1698
print(len(test_data)) #350

1698
350


In [38]:
print(len(all_data))

1698


<h3> 5. Model Load 

In [39]:
#현재 디렉토리 찾기 
# os.getcwd()

%reload_ext autoreload
%autoreload 2

# from train.tt import VIT2
# model = VIT2().to(device)


# from train.VIT_Small import VIT_Small
# model = VIT_Small().to(device)
# model.train()

# from train.VIT_Base import VIT_Base
# model = VIT_Base().to(device)
# model.train()

from train.VIT_B_Deep import VIT_Base_Deep
model = VIT_Base_Deep().to(device)
model.train()

VIT_Base_Deep(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none'

In [40]:
for name, param in model.named_parameters():
    if param.requires_grad==True:
        print(name)

prompt_embeddings
deep_prompt_embeddings
model.head.weight
model.head.bias


In [41]:
print('learnable params:')
sum(p.numel() for p in model.parameters() if p.requires_grad)

learnable params:


53770

In [42]:
#loss label smooth
def loss_fn(outputs, targets):
    if len(targets.shape) == 1:
        return F.cross_entropy(outputs, targets)
    else:
        return torch.mean(torch.sum(-targets * F.log_softmax(outputs, dim=1), dim=1))

def label_smooth_loss_fn(outputs, targets, epsilon=0.1):
    onehot = F.one_hot(targets, 10).float().to(device)
    targets = (1 - epsilon) * onehot + torch.ones(onehot.shape).to(device) * epsilon / 1000
    return loss_fn(outputs, targets)

In [43]:
def func_eval(Model,data_iter,loss):
    with torch.no_grad():
      Model.eval()
      n_total, n_correct = 0,0
      loss_val_sum=0
      print("(Train or Validation) Data Testing....\n")
      for imgs, labels in iter(data_iter):
        imgs, labels = imgs.to(device), labels.to(device)
        
        model_pred=Model(imgs)

        loss_out=loss(model_pred,labels)
        loss_val_sum+=loss_out

        _, y_pred=torch.max(model_pred.data,1)
        n_correct+=(y_pred==labels).sum().item()
        n_total+=imgs.size(0)
      val_accr=(n_correct/n_total)
      Model.train()
      loss_val_avg=loss_val_sum/len(data_iter)
    print("Testing Done.\n")
    return val_accr,loss_val_avg
    
def get_submission(Model,data_iter,epoch,fold=None):
  with torch.no_grad():
    Model.eval()
    pred_label=[]
    print("Final Testing....\n")
    for imgs in iter(test_iter):
      model_pred=Model(imgs.to(device))

      _, y_pred=torch.max(model_pred.data,1)
      pred_label.extend(y_pred.tolist())

  Model.train()

  ##Cross-validation##
  # submission = pd.read_csv('./dataset/test_answer_sample_.csv', encoding = 'utf-8')
  # submission['prediction'] = pred_label
  # submission.to_csv('./submission_VIT_small'+str(fold)+'_'+str(epoch)+'.csv', index = False)

  ##TRAIN-TEST##
  submission = pd.read_csv('./dataset/test_answer_sample_.csv', encoding = 'utf-8') #index_col = 0
  submission['answer value'] = pred_label
  submission.to_csv('./submission_VIT_deep_base16'+'_'+str(epoch)+'.csv', index = False)

<h3> Train

<h2> cross validation 

In [44]:
folds=StratifiedKFold(n_splits=4,shuffle=True)
scaler = torch.cuda.amp.GradScaler()
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)
##Loss Function 택1
loss=label_smooth_loss_fn
# loss = loss_fn 
f = open("./vit_deep_base16.txt", 'w')



In [26]:
for epoch in range(EPOCHS) :
    
  loss_val_sum=0
  visual_loss_sum=0 # 확인용 Train Loss

    
  for imgs, labels in tqdm(iter(train_iter)):
    # Cut mix P=0.5
    imgs, labels = imgs.to(device), labels.to(device)

    optimizer.zero_grad()

    model_pred=model(imgs)
    # print('mo',model_pred.shape) #[2,10] 
    # print('la',labels.shape)
    # print(labels[0])
    

    # Label Smoothing
    loss_out = loss(model_pred, labels) 
    # 확인용 Train Loss
    normal_loss_out=loss_fn(model_pred.clone().detach(), labels.clone().detach()) 
    scaler.scale(loss_out).backward()
    scaler.step(optimizer)
    scaler.update()

    loss_val_sum+=loss_out
    visual_loss_sum+=normal_loss_out

  loss_val_avg=loss_val_sum/len(train_iter)
  visual_loss_avg=visual_loss_sum/len(train_iter)
  get_submission(model,test_iter,epoch)

  print("epoch:[%d] train loss smooth:[%.5f] train normal loss:[%.5f]\n"%(epoch,loss_val_avg,visual_loss_avg))
  f.write("epoch:[%d] train loss smooth:[%.5f] train normal loss:[%.5f]\n"%(epoch,loss_val_avg,visual_loss_avg))
  f.flush()
  print("Model Save....\n")
  torch.save({'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict()}, './checkpoint_vit/ViT_epoch_'+str(epoch)+'.tar')
  scheduler.step(loss_val_avg) # LR Scheduler
f.close()

  0%|          | 0/849 [00:00<?, ?it/s]

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Final Testing....

epoch:[0] train loss smooth:[0.59013] train normal loss:[0.59013]

Model Save....



/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


  0%|          | 0/849 [00:00<?, ?it/s]

...

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


  0%|          | 0/849 [00:00<?, ?it/s]

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Final Testing....

epoch:[39] train loss smooth:[0.01379] train normal loss:[0.01379]

Model Save....



/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


In [ ]:
##CROSS-VALIDATION 
print(BATCH_SIZE)
for current_fold,(train_idx, vali_idx) in enumerate(folds.split(all_data,all_data.label_list)):
    
  train_data=torch.utils.data.Subset(all_data,train_idx)
  vali_data=torch.utils.data.Subset(all_data,vali_idx)
    
  train_iter=DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
  vali_iter=DataLoader(vali_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)

  print(f'::::::::: Fold : {current_fold} :::::::::::\n')
  f.write(f'::::::::: Fold : {current_fold} :::::::::::\n')
  # model.train()
  prev_vali_loss=100
  best_vali_loss=100
  flag=0
  over_check=0
  for epoch in range(EPOCHS) :
   
    loss_val_sum=0
    for imgs, labels in iter(train_iter):
        

        imgs, labels = imgs.to(device), labels.to(device)
        imgs, labels_a, labels_b, lam = cutmix_data(imgs, labels)

        
        model_pred=model(imgs)
        # loss_out = loss(model_pred, labels)
         # Label Smoothing + Cutmix
        loss_out = lam * loss(model_pred, labels_a) + (1 - lam) * loss(model_pred, labels_b)
        # 확인용 Train Loss
        normal_loss_out=lam * loss_fn(model_pred.clone().detach(), labels_a.clone().detach()) + (1 - lam) * loss_fn(model_pred.clone().detach(), labels_b.clone().detach())

        scaler.scale(loss_out).backward()
        scaler.step(optimizer)
        scaler.update()

        loss_val_sum+=loss_out
    loss_val_avg=loss_val_sum/len(train_iter)
    vali_accr,vali_loss=func_eval(model,vali_iter,loss_fn)
    if epoch>=0:
      get_submission(model,test_iter,epoch,current_fold)
      print("epoch:[%d] train loss:[%.5f] vali loss:[%.5f] vali_accr:[%.5f]\n"%(epoch,loss_val_avg,vali_loss,vali_accr))
      f.write("epoch:[%d] train loss:[%.5f] vali loss:[%.5f] vali_accr:[%.5f]\n"%(epoch,loss_val_avg,vali_loss,vali_accr))
      print("Model Save....\n")
      torch.save({'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict()}, './checkpoint/'+str(current_fold)+'_VPT_epoch_'+str(epoch)+'.tar')
    else:
      print("epoch:[%d] train loss:[%.5f] vali loss:[%.5f] vali_accr:[%.5f]\n"%(epoch,loss_val_avg,vali_loss,vali_accr))
      f.write("epoch:[%d] train loss:[%.5f] vali loss:[%.5f] vali_accr:[%.5f]\n"%(epoch,loss_val_avg,vali_loss,vali_accr))
    scheduler.step(vali_loss) # LR Scheduler

    if prev_vali_loss<vali_loss: #Stop Train
      flag+=1
      if flag==10 : 
        print("Stop Training...\n")
        break
    if best_vali_loss>vali_loss:
      flag=0
      best_vali_loss=vali_loss
    prev_vali_loss=vali_loss
f.close()

2
::::::::: Fold : 0 :::::::::::



/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(Train or Validation) Data Testing....

Testing Done.

Final Testing....

epoch:[0] train loss:[2.20351] vali loss:[1.75191] vali_accr:[0.86353]

Model Save....



/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current

(Train or Validation) Data Testing....

Testing Done.

Final Testing....

epoch:[1] train loss:[3.98847] vali loss:[3.39132] vali_accr:[0.85412]

Model Save....



...

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current

(Train or Validation) Data Testing....

Testing Done.

Final Testing....

epoch:[4] train loss:[16.42434] vali loss:[19.85637] vali_accr:[0.78353]

Model Save....



/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current

(Train or Validation) Data Testing....

Testing Done.

Final Testing....

epoch:[5] train loss:[13.46696] vali loss:[6.98874] vali_accr:[0.89647]

Model Save....



/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current

In [ ]:
# dataloader_iter = iter(train_iter)
# batch = next(dataloader_iter)
# print(batch)